In [ ]:
import pandas as pd
import numpy as np
from numpy import array
from sklearn.model_selection import train_test_split
import cv2
import matplotlib.pyplot as plt
import time
from keras.utils.np_utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler, CSVLogger

def classTextToInt(row_label):
    switcher = {
        '1': 1,
        '2': 2,
        '3': 3,
        '4': 4,
        '5': 5,
        '6': 6,
        '7': 7,
        '8': 8,
        '9': 9,
        '0': 10,
        'a': 37,
        'b': 38,
        'c': 13,
        'd': 39,
        'e': 40,
        'f': 41,
        'g': 42,
        'h': 43,
        'i': 19,
        'j': 20,
        'k': 21,
        'l': 22,
        'm': 23,
        'n': 44,
        'o': 25,
        'p': 26,
        'q': 45,
        'r': 46,
        's': 29,
        't': 46,
        'u': 31,
        'v': 32,
        'w': 33,
        'x': 34,
        'y': 35,
        'z': 36,
        'A': 11,
        'B': 12,
        'C': 13,
        'D': 14,
        'E': 15,
        'F': 16,
        'G': 17,
        'H': 18,
        'I': 19,
        'J': 20,
        'K': 21,
        'L': 22,
        'M': 23,
        'N': 24,
        'O': 25,
        'P': 26,
        'Q': 27,
        'R': 28,
        'S': 29,
        'T': 30,
        'U': 31,
        'V': 32,
        'W': 33,
        'X': 34,
        'Y': 35,
        'Z': 36,
        'Random': 47
    }
    return switcher.get(row_label, None)

folderToFind = "by_class/"
offsetLength = len(folderToFind)

def imageResize (image, width = None, height = None, inter = cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]
    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))
    resized = cv2.resize(image, dim, interpolation = inter)
    return resized

def imageGray (image):
    imageG = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return imageG

def downScaleImage(path, index):
    image = cv2.imread(path)   
    grayImage = imageGray(image)
    thresh = cv2.adaptiveThreshold(grayImage, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 101, 40)
    resizedImage = imageResize(thresh, height = 64)
    arr = array(resizedImage)
    return arr

SAVE_PATH = "logs/modelsBalancedComplete"
num_classes = 48

X_train = []
Y_train = []
df1 = pd.read_csv('trainingRecords.csv')
for index, row in df1.iterrows():
    if index % 1000 == 0:
        print('Progress:',(index/len(df1))*100, '%                              ', end='\r')
    temp = downScaleImage(row['fileName'], index)
    filePath = row['fileName']
    fileClass = filePath[filePath.find(folderToFind) + offsetLength]
    toAdd = temp.ravel()
    X_train.append(toAdd)
    Y_train.append(classTextToInt(fileClass))

X_train = array(X_train)
Y_train = array(Y_train)
X_train = np.reshape(X_train, (-1, 64, 64, 1))
# Y_train = to_categorical(Y_train, num_classes)

X_test = []
Y_test = []
df1 = pd.read_csv('validationRecords.csv')
for index, row in df1.iterrows():
    if index % 1000 == 0:
        print('Progress:',(index/len(df1))*100, '%                              ', end='\r')
    temp = downScaleImage(row['fileName'], index)
    filePath = row['fileName']
    fileClass = filePath[filePath.find(folderToFind) + offsetLength]
    toAdd = temp.ravel()
    X_test.append(toAdd)
    Y_test.append(classTextToInt(fileClass))

X_test = array(X_test)
Y_test = array(Y_test)
X_test = np.reshape(X_test, (-1, 64, 64, 1))
# Y_test = to_categorical(Y_test, num_classes)



datagen = ImageDataGenerator(rotation_range=15, zoom_range = 0.15, width_shift_range=0.1, height_shift_range=0.1)

nets = 1
model = [0] * nets
tensorboard = [0] * nets
csv_logger = [0] * nets

for j in range(nets):
    model[j] = Sequential()

    model[j].add(Conv2D(32, kernel_size = 3, activation='relu', input_shape = (64, 64, 1)))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(32, kernel_size = 3, activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Dropout(0.4))

    model[j].add(Conv2D(64, kernel_size = 3, activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(64, kernel_size = 3, activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Dropout(0.4))

    model[j].add(Conv2D(128, kernel_size = 4, activation='relu'))
    model[j].add(BatchNormalization())
    model[j].add(Flatten())
    model[j].add(Dropout(0.4))
    model[j].add(Dense(num_classes, activation='softmax'))

    model[j].compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=['accuracy', 'sparse_categorical_accuracy', 'mean_squared_error', 'mean_absolute_error'])

    NAME = "{}-conv-{}-nodes-{}-dense{}".format(7, 48, 1, int(time.time()))
    tensorboard[j] = TensorBoard(log_dir='{}/{}______{}'.format(SAVE_PATH, NAME, j))
    csv_logger[j] = CSVLogger('{}/csvlogs/{}__{}.csv'.format(SAVE_PATH, NAME, j), separator=',', append=False)

    print('Model', j+1, ':')
    model[j].summary()

annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x)
epochs = 30
history = [0] * nets

for j in range(nets):
    X_train2, X_val2, Y_train2, Y_val2 = train_test_split(X_train, Y_train, test_size = 0.2)
    history[j] = model[j].fit_generator(datagen.flow(X_train2,Y_train2, batch_size=64),
        epochs = epochs, steps_per_epoch = X_train2.shape[0]//64,  
        validation_data = (X_val2,Y_val2),
        callbacks=[annealer, tensorboard[j], csv_logger[j]])
#     print("CNN {0:d}: Epochs={1:d}, Train accuracy={2:.5f}, Validation accuracy={3:.5f}".format(
#         j+1,epochs,max(history[j].history['acc']),max(history[j].history['val_acc']) ))

Using TensorFlow backend.


Model 1 : 99.41543722909293 %                               
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        320       
_________________________________________________________________
batch_normalization (BatchNo (None, 62, 62, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 60, 60, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 60, 60, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        25632     
_________________________________________________________________
batch_normalization_2 (Batch (None, 30, 30, 32)        128       
_____________________________________________________________

In [ ]:
for j in range(nets):
    model[j].evaluate(X_test, Y_test)

for j in range(nets):
    model[j].save("{}/model{0:d}.hdf5".format(SAVE_PATH, j+1))